# Defining Azure Networking Components with Terraform

Azure offers a wide range of networking services and features. To leverage the networking capabilities of Azure while adhering to Infrastructure as Code (IaC) principles, Terraform comes into play as an invaluable tool. This combination allows users to programmatically define, configure, and manage Azure networking components.

In this lesson, you will learn to leverage Terraform to define, deploy, and manage networking components such as *virtual network*, *security groups* and more.

## Azure Networking Basics 

Azure Networking serves as the backbone of any Azure-based application or infrastructure, enabling secure communication, resource isolation and efficient data transfer. It comprises several key components, each with its role and significance. In this section, we will look in detail at some of these components.

### Virtual Network (VNet)

> A *Virtual Network (VNet)* is the fundamental building block of your Azure network infrastructure. It's a logically isolated network that allows you to organize and control communication between Azure resources. 

Key features of VNets include:

- **Address Space**: When creating a VNet, you define its address space, which is a range of private IP addresses. This address space is used to allocate IP addresses to resources within the VNet.
- **Subnets**: VNets can be divided into one or more subnets, each with its address space. Subnets help organize resources and implement network security policies at a more granular level.
- **Peering**: VNets can be peered with each other to allow communication between resources in different VNets, even across different Azure regions

### Subnet

> A *Subnet* is a subdivision of a Virtual Network and is used to group resources logically. 

Here are some key aspects of subnets:

- **Resource Placement**: When you create resources in a VNet, you place them in specific subnets based on their role or function
- **Security**: You can associate *Network Security Groups (NSGs)* with subnets to control inbound and outbound traffic to resources within the subnet
- **Communication**: Resources within the same subnet can communicate freely, while NSGs control communication between different subnets or VNets

### Network Security Group (NSG)

> A **Network Security Group (NSG)** acts as a virtual firewall for controlling network traffic to and from Azure resources. NSGs enable you to define security rules that determine which traffic is allowed or denied. 

NSGs consist of inbound and outbound security rules. *Inbound rules* define allowed or denied incoming traffic, while *outbound rules* control outgoing traffic.

### Virtual Network Peering

> *Virtual Network Peering* allows for secure communication between VNets in Azure. 

Key aspects of VNet peering include:

- **Non-Transitive Connectivity**: VNet peering does not provide transitive connectivity. If VNet A is peered with VNet B and VNet B is peered with VNet C, resources in VNet A cannot communicate directly with resources in VNet C through the established peering relationships. Direct peering connections between VNet A and VNet C would be required for communication between their resources.
- **Security**: Peering does not override NSG rules, so you can maintain control over the traffic between peered VNets by configuring NSGs

### Scenario: Web Application Hosting

Imagine you're tasked with deploying a web application in Azure. Your application consists of a web front-end, an application server, and a database server. Each of these components has distinct networking requirements, and Azure networking components help you meet these requirements effectively.

- Virtual Network (VNet):

  - In Azure, you create a Virtual Network (VNet) named `my-app-vnet` with an address space of `10.0.0.0/16`. This VNet serves as the network foundation for your entire application. 

    - The VNet here defines the IP address space in *CIDR notation* (e.g., `10.0.0.0/16`). **CIDR (Classless Inter-Domain Routing)** notation is a method for representing IP addresses and their associated routing prefix. In this case, `10.0.0.0/16` signifies that the VNet has a range of IP addresses from `10.0.0.0` to `10.0.255.255`. The `/16` indicates the *subnet mask*, specifying the number of significant bits in the address space. The `10.0.0.0/16` address space provides a large pool of IP addresses for your resources to use. <br><br>
  
- Subnets:

  - Within the VNet, you create two subnets:
    - `web-subnet` (address space: `10.0.1.0/24`): This subnet will host your web front-end servers
    - `app-db-subnet` (address space: `10.0.2.0/24`): This subnet will host both the application servers and the database server

      - The subnets are defined with their own unique address spaces within the parent VNet. For example, `10.0.1.0/24` for the `web-subnet` means that this subnet has an address space from `10.0.1.0` to `10.0.1.255`, allowing for up to 256 usable IP addresses. Similarly, `10.0.2.0/24` for the `app-db-subnet` indicates its address space from `10.0.2.0` to `10.0.2.255`, accommodating up to 256 IP addresses. <br><br>

- Network Security Groups (NSGs):
  - To secure your resources, you configure Network Security Groups (NSGs) as follows:
    - `web-subnet-nsg` for the `web` subnet: This NSG allows HTTP (port 80) and HTTPS (port 443) traffic and blocks all other incoming traffic
    - `app-db-subnet-nsg` for the `app/db` subnet: This NSG allows communication between the application server and the database server on specific ports (e.g., 1433 for SQL Server) and blocks all other incoming traffic <br><br>
    
- Virtual Machines (VMs):
  - You create Virtual Machines in the respective subnets:
    - Web Front-end VMs in `web-subnet`
    - Application Server and Database Server VMs in `app-db-subnet` <br><br>

- Virtual Network Peering:
  - To enable communication between the web front-end and application server, you do not need to establish Virtual Network Peering between the `web-subnet` and `app-db-subnet`. This is because traffic between subnets within the same VNet can be route from a subnet to another, without the need for peering. Instead, VNet peering is used to connect different Virtual Networks, enabling communication between resources in separate VNets. <br><br>
  
Here's how these Azure networking components work together in this scenario:

- **Web Front-end**: The web front-end VMs in `web-subnet` receive HTTP/HTTPS requests from clients. The NSG associated with `web-subnet` ensures that only HTTP/HTTPS traffic is allowed, providing security.

- **Application Server**: The application server VMs in `app-db-subnet` need to communicate with the database server VMs in the same subnet. The NSG associated with `app-db-subnet` allows this specific communication while blocking other incoming traffic. The Virtual Network Peering ensures secure communication with the web front-end.

- **Database Server**: The database server VMs in `app-db-subnet` receive database queries from the application server. The NSG for `app-db-subnet` allows only the necessary database-related traffic.

By effectively configuring Azure networking components like VNets, subnets, NSGs, and Virtual Network Peering, you can create a secure and well-organized network environment that supports your web application's functionality while maintaining security and isolation between components.

## Defining Azure Networking with Terraform

In this section we will look at how to define Azure Networking components using Terraform. Terraform allows us to create, manage, and automate our Azure network infrastructure in a structured and declarative manner.

Let's start by considering a simplified Terraform configuration file (`main.tf`) that defines a VNet in Azure:

```hcl
provider "azurerm" {
  features {}
}

resource "azurerm_virtual_network" "example" {
  name                = "example-network"
  address_space       = ["10.0.0.0/16"]
  location            = "East US"
  resource_group_name = azurerm_resource_group.example.name
}

resource "azurerm_resource_group" "example" {
  name     = "example-resources"
  location = "East US"
}
```

In this example:
- We specify the Azure provider for Terraform
- We create an Azure Virtual Network with a specified name, address space, location, and resource group
- We also define an Azure Resource Group as a logical container for our resources

Let's break down the key steps in creating a Virtual Network using Terraform.

### Step 1: Resource Configuration for a VNet

In Terraform, we define the configuration for our Virtual Network using the `azurerm_virtual_network` resource block. This block specifies attributes such as the VNet's name, address space, location, and other essential details.

> **Address space** planning is a critical step. It involves determining the IP address range for your VNet. It's essential to choose an address space that doesn't overlap with other networks to avoid IP conflicts.

### Step 2: Associating Subnets with the VNet

Subnets can be associated with the VNet using the `azurerm_subnet` resource block. This allows us to create multiple subnets within the VNet for resource organization and security boundaries. Let's look at an example subnet configuration:

```hcl
resource "azurerm_subnet" "example" {
  name                 = "example-subnet"
  resource_group_name  = azurerm_resource_group.example.name
  virtual_network_name = azurerm_virtual_network.example.name
  address_prefixes     = ["10.0.1.0/24"]
}
```
In this example:
- `name`: This parameter sets the name of the subnet as `example-subnet`
- `resource_group_name`: It specifies the Azure Resource Group where the subnet will be created. In this case, it's the same resource group as defined earlier.
- `virtual_network_name`: This parameter associates the subnet with a Virtual Network, ensuring it's created within that network
- `address_prefixes`: This defines the IP address range for the subnet. `10.0.1.0/24` means it can have IP addresses from `10.0.1.0` to `10.0.1.255` within the Virtual Network's larger address space.

### Step 3: Configuring Network Security Groups (NSGs) with Terraform

To enhance security, we define Network Security Groups (NSGs) using Terraform's `azurerm_network_security_group` resource block. NSGs allow us to control inbound and outbound traffic to resources within subnets. For example:

```hcl
resource "azurerm_network_security_group" "example" {
  name                = "example-nsg"
  location            = azurerm_resource_group.example.location
  resource_group_name = azurerm_resource_group.example.name

  security_rule {
    name                       = "allow-ssh"
    priority                   = 1001
    direction                  = "Inbound"
    access                     = "Allow"
    protocol                   = "Tcp"
    source_port_range          = "*"
    destination_port_range     = "22"
    source_address_prefix      = "*"
    destination_address_prefix = "*"
  }
}
```
This Terraform configuration creates an Azure Network Security Group named `example-nsg` within the specified resource group and location. It also defines a security rule named `allow-ssh` that allows inbound TCP traffic on port 22 from any source to any destination within the NSG. Now let's break this example down:

- `name`: This parameter sets the name of the NSG to `example-nsg`

- `location`: It specifies the location (Azure region) where the NSG will be created. In this case, it references the location of the Azure Resource Group (`azurerm_resource_group.example.location`), ensuring the NSG is created in the same region as the resource group.

- `resource_group_name`: This parameter defines the name of the Azure Resource Group where the NSG will be placed. It references `azurerm_resource_group.example.name`, indicating that the NSG will be created within the same resource group defined earlier in your Terraform configuration.

- `security_rule`: Inside the NSG resource block, we define a security rule that configures the network traffic rules for this NSG
  - `name`: Sets the name of the security rule as `allow-ssh`

  - `priority`: This value specifies the rule's priority, which determines the order in which rules are applied. The lower the numerical value of the `priority`, the higher the precedence of the rule.  In this case, it's set to `1001`. This means that this security rule has a relatively high priority within the NSG. When network traffic hits this NSG, the rule `allow-ssh` with priority `1001` will be evaluated before rules with higher numerical priorities.

  - `direction`: Specifies the direction of network traffic to which the rule applies. Here, it's set to `Inbound`, indicating it applies to incoming traffic.

  - `access`: Defines whether the rule allows or denies the specified traffic. It's set to `Allow`, meaning it permits the specified traffic.

  - `protocol`: Specifies the network protocol for the rule. In this example, it's set to `Tcp`, indicating it allows TCP traffic. TCP traffic refers to the communication of data over a computer network using the TCP protocol. TCP is one of the core protocols of the Internet Protocol (IP) suite and is responsible for ensuring reliable and ordered data transmission between devices on a network.

  - `source_port_range` and `destination_port_range`: These parameters define the source and destination port ranges for the traffic. In this case, the source port range is set to `*` (meaning any port), and the destination port range is set to `22`, which is the port commonly used for SSH. SSH is a network protocol and cryptographic technology used to secure communication over a computer network.

  - `source_address_prefix` and `destination_address_prefix`: These parameters define the source and destination IP address ranges for the traffic. `*` means it allows traffic from any source to any destination.


## Hands-On: Create Necessary Azure Networking Service for an AKS Cluster

In this hands-on example we will learn how to create the necessary Azure networking services for an Azure Kubernetes Service (AKS) cluster using Terraform. In this example, we'll set up the Azure Virtual Network (VNet), Subnet, and Network Security Group (NSG) for the AKS cluster. Please note that this is a simplified example, and production environments may require additional configurations and security considerations.

### Step 1: Create the Terraform Project Folder

Start by creating a directory for your Terraform project. This will contain both the main configuration and the module directory.

```shell
mkdir aks-networking-project
cd aks-networking-project
```

### Step 2: Create the Terraform Module Directory

Inside your project folder, create a directory for the Terraform module. You can name it something like `aks-networking-module`.

```shell
mkdir aks-networking-module
cd aks-networking-module
```

### Step 3: Create the Resource Group (`resource-group.tf`)

In the `aks-networking-module` directory, create a file named `resource-group.tf` to define the resource group creation:

```hcl
resource "azurerm_resource_group" "aks" {
  name     = "aks-resources"
  location = "UK South" # Change this to your desired region.
}
```

### Step 4: Create the Virtual Network (VNet) (`virtual-network.tf`)

Create a file named `virtual-network.tf` in the `aks-networking-module` directory to define the Virtual Network creation:

```hsl
resource "azurerm_virtual_network" "aks_vnet" {
  name                = "aks-vnet"
  address_space       = ["10.0.0.0/16"]
  location            = azurerm_resource_group.aks.location
  resource_group_name = azurerm_resource_group.aks.name
}
```
 
### Step 5: Create a Subnet within the VNet (`subnet.tf`)

In the `aks-networking-module` directory, create a file named `subnet.tf` to define the Subnet creation:

```hsl
resource "azurerm_subnet" "aks_subnet" {
  name                 = "aks-subnet"
  resource_group_name  = azurerm_resource_group.aks.name
  virtual_network_name = azurerm_virtual_network.aks_vnet.name
  address_prefixes     = ["10.0.1.0/24"]
}
```

### Step 6: Create a Network Security Group (NSG) (`network-security-group.tf`)

In the `aks-networking-module` directory, create a file named `network-security-group.tf` to define the NSG creation:

```hsl
resource "azurerm_network_security_group" "aks_nsg" {
  name                = "aks-nsg"
  location            = azurerm_resource_group.aks.location
  resource_group_name = azurerm_resource_group.aks.name
}
```

### Step 7: Define a Security Rule for NSG (`security-rule.tf`)

In the `aks-networking-module` directory, create a file named `security-rule.tf` to define the security rule within the NSG:

```hsl
resource "azurerm_network_security_rule" "allow_http" {
  name                        = "allow-http"
  resource_group_name         = azurerm_resource_group.aks.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
  priority                    = 1001
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "80"
  source_address_prefix       = "*"
  destination_address_prefix  = "*"
}

resource "azurerm_network_security_rule" "allow_https" {
  name                        = "allow-https"
  resource_group_name         = azurerm_resource_group.aks.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
  priority                    = 1002
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "443"
  source_address_prefix       = "*"
  destination_address_prefix  = "*"
}

resource "azurerm_network_security_rule" "allow_node_ports" {
  name                        = "allow-node-ports"
  resource_group_name         = azurerm_resource_group.aks.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
  priority                    = 1003
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "30000-32767"
  source_address_prefix       = "*"
  destination_address_prefix  = "*"
}
```

This example creates three separate security rules: one for port 80, one for port 443, and one for the range of ports from 30000 to 32767. These are commonly used ports by AKS clusters.

### Step 8: Initialize the Terraform Module

Inside the `aks-networking-module` directory run `terraform init` to initialize your Terraform module.

### Step 9: Create Main Configuration File (`main.tf`)

Now, go back to the root directory (`aks-networking-project`) and create your main configuration file (`main.tf`) for initializing the module:

```hcl
terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "=3.0.0"
    }
  }
}

provider "azurerm" {
  features {}
  client_id       = "your-client-id"
  client_secret   = "your-client-secret"
  subscription_id = "your-subscription-id"
  tenant_id       = "your-tenant-id"
}

module "aks_networking" {
  source              = "./aks-networking-module" # Path to your module directory
}
```

### Step 10: Initialize and Apply the Configuration

Within your root directory `aks-networking-project` initialize the Terraform configuration, then plan and apply the changes:

```shell
terraform init
terraform plan
terraform apply
```

When running `terraform plan` you should see 7 resources planned to be added. 

After running `terraform apply` follow the prompts and type **yes** to apply the configuration. Congratulations you have successfully provisioned the Azure networking components needed to support an AKS cluster!

### Step 11: Clean Up Resources 

If you want to delete the resources created by Terraform, you can use the following command: `terraform destroy`. Confirm the deletion by typing **yes** when prompted. This will remove all the resources created in the `aks-resources` resource group.

## Key Takeaways

- VNets are fundamental to Azure networking, providing logical isolation for resources. They enable precise control over communication, allowing you to segment your Azure infrastructure into distinct network environments.
- Subnets are indispensable for organizing VNets. They optimize IP address allocation, making resource management efficient and scalable by partitioning VNets into smaller, manageable segments.
- NSGs act serving as firewalls that regulate inbound and outbound traffic. By configuring NSG security rules, you can enforce fine-grained access control and protect your resources from unauthorized access.
- VPC peering allows communication between different VNets by facilitating data transfer across network boundaries
- Terraform allows you to provision Azure networking resources as code, enabling automation, version control, and reproducibility
- Define Azure networking resources using Terraform resource blocks (e.g., `azurerm_virtual_network`, `azurerm_subnet`). Specify their configurations, such as name, address space, and security rules.